In [45]:
from importlib import reload
import features
reload(features)

<module 'features' from 'C:\\Users\\baudo\\Desktop\\PFE\\Scripts\\features.py'>

In [46]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import accuracy_score
from sklearn import manifold

from features import ennumerate, load, build
from copy import deepcopy

In [47]:
availablesFeatures = ennumerate()
availablesFeatures

['darkduration',
 'darkspeed',
 'deviationCos',
 'deviationSin',
 'disappearBearingCos',
 'disappearBearingSin',
 'disappearDistance',
 'disappearEdgeRatio',
 'reappearBearingCos',
 'reappearBearingSin',
 'reappearDistance',
 'reappearEdgeRatio']

In [48]:
data = build([load(name) for name in availablesFeatures])
data

,darkduration,darkspeed,deviationCos,deviationSin,disappearBearingCos,disappearBearingSin,disappearDistance,disappearEdgeRatio,reappearBearingCos,reappearBearingSin,reappearDistance,reappearEdgeRatio
0,90235.0,0.424231,-0.744312,0.667833,-0.825126,-0.564949,12.604492,0.000000,0.012021,-0.999928,8.381199,0.300000
1,20997.0,4.437835,0.893371,0.449319,-0.451221,-0.892412,25.334982,0.000000,0.521523,-0.853237,27.837232,0.179608
2,6849.0,9.238654,0.165048,0.986286,0.522172,-0.852840,27.830877,0.151558,0.413808,-0.910364,14.780363,0.000000
3,108566.0,0.000290,0.327218,0.944949,0.011404,-0.999935,8.388079,0.167841,0.011847,-0.999930,8.382786,0.350000
4,57612.0,0.003477,0.245307,0.969445,0.007340,-0.999973,8.401267,0.132622,0.013754,-0.999905,8.391414,0.456947
...,...,...,...,...,...,...,...,...,...,...,...,...
16599,276577.0,0.061721,0.644124,0.764921,0.182476,0.983210,103.136296,0.962535,0.226096,0.974105,104.210692,0.982315
16600,1963701.0,0.008930,-0.943223,0.332161,0.226163,0.974090,104.216466,0.982305,0.185001,0.982738,102.355018,0.875247
16601,5970.0,7.125487,0.999986,0.005236,0.185001,0.982738,102.355018,0.875247,0.240749,0.970587,95.963557,0.877994
16602,7458525.0,0.005457,-0.981627,0.190809,0.240811,0.970572,95.889614,0.879171,0.188737,0.982028,102.062970,0.819963


In [31]:
scaler = StandardScaler()

train_data = data.copy(deep=True)  # train_data correspond au datatset

train_data_label = train_data["label"]            # train_data_label correspond aux labels 
full_x = train_data.drop("label", axis = 1)

scaler.fit(full_x)

full_train_data_x = scaler.transform(full_x)
full_train_data_x = pd.DataFrame(full_train_data_x , columns = full_x.columns)


KeyError: 'label'

# Algo Active SVM

In [ ]:
def active_svm(full_train_data_x, train_data_label):
    n_cycles = 3
    first_batch = 15
    batch_s = 7
    dsize = len(full_train_data_x)

    train_data_x = full_train_data_x.copy(deep = True)
    
    tr_index = np.random.randint(0, dsize, first_batch)         # first first_batch labels to Query 
    tr_data_x = train_data_x.iloc[tr_index]
    tr_data_label = train_data_label.iloc[tr_index] 
    
    while len(tr_data_label.unique())<1 :
        tr_index = np.random.randint(0, dsize, first_batch)         # first first_batch labels to Query 
        tr_data_x = train_data_x.iloc[tr_index]
        tr_data_label = train_data_label.iloc[tr_index]             # tr_data_label contains all the known label
                                # tr_data_x contains the features from the points we know the label

    train_data_x = train_data_x.drop(index = tr_index)          # on take out the points from which we know the label


    params = {'gamma':[1/dsize, 0.01,0.1,1,10]}          # dictionnary for the grid search to find the optimal value of gamma  
                                                         # based on the first first_batch samples

    rbf_svc=SVC(kernel='rbf', max_iter=200000)
    r_svc_active = GridSearchCV(rbf_svc, params)    # grid search to find an optimal value of gamma

    avg_test_accu = np.zeros(n_cycles)

    # while avg_test_accu[-1]<0.95 :
    for i in range(n_cycles) :

        r_svc_active.fit(tr_data_x,tr_data_label)  # fit the RBF SVM models on the known data points
        
        distances_index = train_data_x.index[np.random.randint(0, len(train_data_x), batch_s)]# keep the index to find the datapooints in the other DataFrames
        
        new_train_x  =  train_data_x.loc[distances_index]   # get the features of the selected points
        
        '''
        Ici il faut inclure le code pour faire la requête de label à l'expert
        '''

        new_train_lab = train_data_label.loc[distances_index]    
        train_data_x= train_data_x.drop(index = distances_index) # delete the selected points from train_data_x since 
                                                                 # we know their label
        tr_data_x = pd.concat([tr_data_x,new_train_x],axis=0)           # on ajoute les nouveaux points connus à tr_data
        tr_data_label = pd.concat([tr_data_label,new_train_lab],axis=0) # on ajoute les nouveaux labels connus également

        pred_lab = r_svc_active.predict(full_train_data_x)    # on fait une prédiction sur tout le dataset pour avoir 
                                                              #l'accuracy       
        avg_test_accu[i]=accuracy_score(train_data_label,pred_lab)
        #print(avg_test_accu)
    return r_svc_active